In [1]:
import re
import random

data_path = "model/human_text.txt"
data_path2 = "model/robot_text.txt"
# Defining lines as a list of each line
with open(data_path, 'r', encoding='utf-8') as f:
  lines = f.read().split('\n')
with open(data_path2, 'r', encoding='utf-8') as f:
  lines2 = f.read().split('\n')
lines = [re.sub(r"\[\w+\]",'hola',line) for line in lines]
lines = [" ".join(re.findall(r"\w+",line)) for line in lines]
lines2 = [re.sub(r"\[\w+\]",'',line) for line in lines2]
lines2 = [" ".join(re.findall(r"\w+",line)) for line in lines2]
# Grouping lines by response pair
pairs = list(zip(lines,lines2))
#random.shuffle(pairs)

In [3]:
import numpy as np

input_docs = []
target_docs = []
input_tokens = set()
target_tokens = set()
for line in pairs:
  input_doc, target_doc = line[0], line[1]
  # Appending each input sentence to input_docs
  input_docs.append(input_doc)
  # Splitting words from punctuation  
  target_doc = " ".join(re.findall(r"[\w']+|[^\s\w]", target_doc))
  # Redefine target_doc below and append it to target_docs
  target_doc = '<START> ' + target_doc + ' <END>'
  target_docs.append(target_doc)
  
  # Now we split up each sentence into words and add each unique word to our vocabulary set
  for token in re.findall(r"[\w']+|[^\s\w]", input_doc):
    if token not in input_tokens:
      input_tokens.add(token)
  for token in target_doc.split():
    if token not in target_tokens:
      target_tokens.add(token)
input_tokens = sorted(list(input_tokens))
target_tokens = sorted(list(target_tokens))
num_encoder_tokens = len(input_tokens)
num_decoder_tokens = len(target_tokens)

input_features_dict = dict(
    [(token, i) for i, token in enumerate(input_tokens)])
target_features_dict = dict(
    [(token, i) for i, token in enumerate(target_tokens)])

reverse_input_features_dict = dict(
    (i, token) for token, i in input_features_dict.items())
reverse_target_features_dict = dict(
    (i, token) for token, i in target_features_dict.items())


max_encoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", input_doc)) for input_doc in input_docs])
max_decoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", target_doc)) for target_doc in target_docs])

encoder_input_data = np.zeros(
    (len(input_docs), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_docs), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_docs), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for line, (input_doc, target_doc) in enumerate(zip(input_docs, target_docs)):
    for timestep, token in enumerate(re.findall(r"[\w']+|[^\s\w]", input_doc)):
        #Assign 1. for the current line, timestep, & word in encoder_input_data
        encoder_input_data[line, timestep, input_features_dict[token]] = 1.
    
    for timestep, token in enumerate(target_doc.split()):
        decoder_input_data[line, timestep, target_features_dict[token]] = 1.
        if timestep > 0:
            decoder_target_data[line, timestep - 1, target_features_dict[token]] = 1.

In [4]:
print(pairs[:5])
print(input_docs[:5])

[('hola Buenos días', 'Buenos Días En que puedo servirle'), ('tienes domicilio', 'Claro que si Que deseas pedir a domicilio'), ('que cuesta la arepa de huevo', 'La arepa tiene un costo de 2000 pesos sin incluir el domicilio'), ('y vendes chocolatico caliente', 'Claro el chocolate un costo de 1000 sin leche'), ('Y con leche', 'con lehe tiene un costo de 2000 pesos')]
['hola Buenos días', 'tienes domicilio', 'que cuesta la arepa de huevo', 'y vendes chocolatico caliente', 'Y con leche']


In [5]:
from tensorflow import keras
from keras.layers import Input, LSTM, Dense
from keras.models import Model
#Dimensionality
dimensionality = 256
#The batch size and number of epochs
batch_size = 10
epochs = 600
#Encoder
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_lstm = LSTM(dimensionality, return_state=True)
encoder_outputs, state_hidden, state_cell = encoder_lstm(encoder_inputs)
encoder_states = [state_hidden, state_cell]
#Decoder
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(dimensionality, return_sequences=True, return_state=True)
decoder_outputs, decoder_state_hidden, decoder_state_cell = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

#Model
training_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
#Compiling
training_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'], sample_weight_mode='temporal')
#Training
training_model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size = batch_size, epochs = epochs, validation_split = 0.2)
training_model.save('model/training_model.h5')

Epoch 1/600
21/21 [==============================] - 8s 155ms/step - loss: 1.4753 - accuracy: 0.0251 - val_loss: 1.3201 - val_accuracy: 0.0326
Epoch 2/600
21/21 [==============================] - 1s 59ms/step - loss: 1.2475 - accuracy: 0.0341 - val_loss: 1.3236 - val_accuracy: 0.0296
Epoch 3/600
21/21 [==============================] - 1s 59ms/step - loss: 1.3189 - accuracy: 0.0357 - val_loss: 1.3183 - val_accuracy: 0.0326
Epoch 4/600
21/21 [==============================] - 1s 59ms/step - loss: 1.2907 - accuracy: 0.0351 - val_loss: 1.3197 - val_accuracy: 0.0337
Epoch 5/600
21/21 [==============================] - 1s 67ms/step - loss: 1.2705 - accuracy: 0.0361 - val_loss: 1.3158 - val_accuracy: 0.0337
Epoch 6/600
21/21 [==============================] - 1s 60ms/step - loss: 1.2546 - accuracy: 0.0396 - val_loss: 1.3126 - val_accuracy: 0.0352
Epoch 7/600
21/21 [==============================] - 2s 85ms/step - loss: 1.1833 - accuracy: 0.0393 - val_loss: 1.3247 - val_accuracy: 0.0331
Epoch

21/21 [==============================] - 1s 57ms/step - loss: 0.8741 - accuracy: 0.1051 - val_loss: 1.3466 - val_accuracy: 0.0612
Epoch 59/600
21/21 [==============================] - 1s 58ms/step - loss: 0.7911 - accuracy: 0.1083 - val_loss: 1.3429 - val_accuracy: 0.0602
Epoch 60/600
21/21 [==============================] - 1s 58ms/step - loss: 0.7830 - accuracy: 0.1088 - val_loss: 1.3433 - val_accuracy: 0.0632
Epoch 61/600
21/21 [==============================] - 1s 63ms/step - loss: 0.9001 - accuracy: 0.1076 - val_loss: 1.3392 - val_accuracy: 0.0586
Epoch 62/600
21/21 [==============================] - 1s 57ms/step - loss: 0.8514 - accuracy: 0.1080 - val_loss: 1.3355 - val_accuracy: 0.0648
Epoch 63/600
21/21 [==============================] - 1s 63ms/step - loss: 0.8329 - accuracy: 0.1086 - val_loss: 1.3249 - val_accuracy: 0.0678
Epoch 64/600
21/21 [==============================] - 1s 65ms/step - loss: 0.7718 - accuracy: 0.1121 - val_loss: 1.3317 - val_accuracy: 0.0704
Epoch 65/600

21/21 [==============================] - 1s 58ms/step - loss: 0.5292 - accuracy: 0.1777 - val_loss: 1.2823 - val_accuracy: 0.0831
Epoch 116/600
21/21 [==============================] - 1s 58ms/step - loss: 0.5461 - accuracy: 0.1745 - val_loss: 1.2888 - val_accuracy: 0.0836
Epoch 117/600
21/21 [==============================] - 1s 59ms/step - loss: 0.4956 - accuracy: 0.1719 - val_loss: 1.2799 - val_accuracy: 0.0862
Epoch 118/600
21/21 [==============================] - 1s 57ms/step - loss: 0.5380 - accuracy: 0.1822 - val_loss: 1.2818 - val_accuracy: 0.0857
Epoch 119/600
21/21 [==============================] - 1s 57ms/step - loss: 0.4969 - accuracy: 0.1740 - val_loss: 1.2915 - val_accuracy: 0.0821
Epoch 120/600
21/21 [==============================] - 1s 57ms/step - loss: 0.4721 - accuracy: 0.1741 - val_loss: 1.3003 - val_accuracy: 0.0847
Epoch 121/600
21/21 [==============================] - 1s 57ms/step - loss: 0.5000 - accuracy: 0.1807 - val_loss: 1.2928 - val_accuracy: 0.0831
Epoch 

Epoch 172/600
21/21 [==============================] - 2s 72ms/step - loss: 0.3921 - accuracy: 0.2085 - val_loss: 1.3059 - val_accuracy: 0.0831
Epoch 173/600
21/21 [==============================] - 1s 71ms/step - loss: 0.3899 - accuracy: 0.2143 - val_loss: 1.3080 - val_accuracy: 0.0847
Epoch 174/600
21/21 [==============================] - 1s 64ms/step - loss: 0.3772 - accuracy: 0.2020 - val_loss: 1.2879 - val_accuracy: 0.0847
Epoch 175/600
21/21 [==============================] - 2s 79ms/step - loss: 0.3733 - accuracy: 0.2007 - val_loss: 1.3102 - val_accuracy: 0.0847
Epoch 176/600
21/21 [==============================] - 1s 59ms/step - loss: 0.3625 - accuracy: 0.2081 - val_loss: 1.3164 - val_accuracy: 0.0847
Epoch 177/600
21/21 [==============================] - 1s 65ms/step - loss: 0.3850 - accuracy: 0.2081 - val_loss: 1.3061 - val_accuracy: 0.0857
Epoch 178/600
21/21 [==============================] - 2s 78ms/step - loss: 0.3810 - accuracy: 0.2061 - val_loss: 1.3164 - val_accuracy:

Epoch 229/600
21/21 [==============================] - 1s 57ms/step - loss: 0.2938 - accuracy: 0.2183 - val_loss: 1.3149 - val_accuracy: 0.0836
Epoch 230/600
21/21 [==============================] - 1s 60ms/step - loss: 0.3001 - accuracy: 0.2186 - val_loss: 1.3347 - val_accuracy: 0.0841
Epoch 231/600
21/21 [==============================] - 1s 58ms/step - loss: 0.3054 - accuracy: 0.2175 - val_loss: 1.3222 - val_accuracy: 0.0831
Epoch 232/600
21/21 [==============================] - 1s 63ms/step - loss: 0.2929 - accuracy: 0.2178 - val_loss: 1.3288 - val_accuracy: 0.0852
Epoch 233/600
21/21 [==============================] - 1s 63ms/step - loss: 0.3245 - accuracy: 0.2186 - val_loss: 1.3223 - val_accuracy: 0.0841
Epoch 234/600
21/21 [==============================] - 1s 59ms/step - loss: 0.2765 - accuracy: 0.2192 - val_loss: 1.3212 - val_accuracy: 0.0841
Epoch 235/600
21/21 [==============================] - 1s 58ms/step - loss: 0.3049 - accuracy: 0.2111 - val_loss: 1.3352 - val_accuracy:

...

Epoch 343/600
21/21 [==============================] - 1s 58ms/step - loss: 0.2421 - accuracy: 0.2298 - val_loss: 1.3607 - val_accuracy: 0.0826
Epoch 344/600
21/21 [==============================] - 1s 59ms/step - loss: 0.2329 - accuracy: 0.2414 - val_loss: 1.3674 - val_accuracy: 0.0801
Epoch 345/600
21/21 [==============================] - 1s 60ms/step - loss: 0.2422 - accuracy: 0.2239 - val_loss: 1.3771 - val_accuracy: 0.0816
Epoch 346/600
21/21 [==============================] - 1s 58ms/step - loss: 0.2468 - accuracy: 0.2424 - val_loss: 1.3692 - val_accuracy: 0.0841
Epoch 347/600
21/21 [==============================] - 1s 64ms/step - loss: 0.2169 - accuracy: 0.2264 - val_loss: 1.3679 - val_accuracy: 0.0831
Epoch 348/600
21/21 [==============================] - 1s 66ms/step - loss: 0.2137 - accuracy: 0.2240 - val_loss: 1.3613 - val_accuracy: 0.0831
Epoch 349/600
21/21 [==============================] - 1s 60ms/step - loss: 0.2360 - accuracy: 0.2307 - val_loss: 1.3289 - val_accuracy:

Epoch 400/600
21/21 [==============================] - 1s 62ms/step - loss: 0.2072 - accuracy: 0.2313 - val_loss: 1.3746 - val_accuracy: 0.0831
Epoch 401/600
21/21 [==============================] - 1s 58ms/step - loss: 0.2195 - accuracy: 0.2324 - val_loss: 1.3781 - val_accuracy: 0.0796
Epoch 402/600
21/21 [==============================] - 1s 58ms/step - loss: 0.2026 - accuracy: 0.2297 - val_loss: 1.3811 - val_accuracy: 0.0775
Epoch 403/600
21/21 [==============================] - 1s 59ms/step - loss: 0.2640 - accuracy: 0.2438 - val_loss: 1.3754 - val_accuracy: 0.0790
Epoch 404/600
21/21 [==============================] - 1s 59ms/step - loss: 0.2146 - accuracy: 0.2356 - val_loss: 1.4057 - val_accuracy: 0.0780
Epoch 405/600
21/21 [==============================] - 1s 59ms/step - loss: 0.2223 - accuracy: 0.2438 - val_loss: 1.3773 - val_accuracy: 0.0826
Epoch 406/600
21/21 [==============================] - 1s 59ms/step - loss: 0.2099 - accuracy: 0.2323 - val_loss: 1.3842 - val_accuracy:

Epoch 457/600
21/21 [==============================] - 1s 59ms/step - loss: 0.1804 - accuracy: 0.2400 - val_loss: 1.3869 - val_accuracy: 0.0831
Epoch 458/600
21/21 [==============================] - 1s 58ms/step - loss: 0.1852 - accuracy: 0.2439 - val_loss: 1.4092 - val_accuracy: 0.0734
Epoch 459/600
21/21 [==============================] - 1s 59ms/step - loss: 0.1670 - accuracy: 0.2332 - val_loss: 1.3597 - val_accuracy: 0.0872
Epoch 460/600
21/21 [==============================] - 1s 58ms/step - loss: 0.1571 - accuracy: 0.2276 - val_loss: 1.3532 - val_accuracy: 0.0872
Epoch 461/600
21/21 [==============================] - 1s 59ms/step - loss: 0.1974 - accuracy: 0.2522 - val_loss: 1.3534 - val_accuracy: 0.0872
Epoch 462/600
21/21 [==============================] - 1s 58ms/step - loss: 0.2024 - accuracy: 0.2483 - val_loss: 1.3614 - val_accuracy: 0.0852
Epoch 463/600
21/21 [==============================] - 1s 59ms/step - loss: 0.1619 - accuracy: 0.2332 - val_loss: 1.3574 - val_accuracy:

Epoch 514/600
21/21 [==============================] - 1s 58ms/step - loss: 0.1830 - accuracy: 0.2532 - val_loss: 1.3386 - val_accuracy: 0.0898
Epoch 515/600
21/21 [==============================] - 1s 58ms/step - loss: 0.1619 - accuracy: 0.2518 - val_loss: 1.3460 - val_accuracy: 0.0892
Epoch 516/600
21/21 [==============================] - 1s 58ms/step - loss: 0.1630 - accuracy: 0.2418 - val_loss: 1.3866 - val_accuracy: 0.0831
Epoch 517/600
21/21 [==============================] - 1s 57ms/step - loss: 0.1565 - accuracy: 0.2506 - val_loss: 1.3926 - val_accuracy: 0.0801
Epoch 518/600
21/21 [==============================] - 1s 58ms/step - loss: 0.1653 - accuracy: 0.2498 - val_loss: 1.3496 - val_accuracy: 0.0877
Epoch 519/600
21/21 [==============================] - 1s 58ms/step - loss: 0.1562 - accuracy: 0.2479 - val_loss: 1.4060 - val_accuracy: 0.0821
Epoch 520/600
21/21 [==============================] - 1s 57ms/step - loss: 0.1708 - accuracy: 0.2545 - val_loss: 1.3938 - val_accuracy:

Epoch 571/600
21/21 [==============================] - 1s 58ms/step - loss: 0.1391 - accuracy: 0.2505 - val_loss: 1.3974 - val_accuracy: 0.0841
Epoch 572/600
21/21 [==============================] - 1s 58ms/step - loss: 0.1322 - accuracy: 0.2378 - val_loss: 1.3770 - val_accuracy: 0.0877
Epoch 573/600
21/21 [==============================] - 1s 58ms/step - loss: 0.1268 - accuracy: 0.2508 - val_loss: 1.4217 - val_accuracy: 0.0816
Epoch 574/600
21/21 [==============================] - 1s 58ms/step - loss: 0.1503 - accuracy: 0.2565 - val_loss: 1.4018 - val_accuracy: 0.0806
Epoch 575/600
21/21 [==============================] - 1s 58ms/step - loss: 0.1363 - accuracy: 0.2528 - val_loss: 1.3796 - val_accuracy: 0.0857
Epoch 576/600
21/21 [==============================] - 1s 58ms/step - loss: 0.1389 - accuracy: 0.2539 - val_loss: 1.3914 - val_accuracy: 0.0826
Epoch 577/600
21/21 [==============================] - 1s 58ms/step - loss: 0.1326 - accuracy: 0.2475 - val_loss: 1.3964 - val_accuracy:

In [ ]:
from keras.models import load_model
training_model = load_model('model/training_model.h5')
encoder_inputs = training_model.input[0]
encoder_outputs, state_h_enc, state_c_enc = training_model.layers[2].output
encoder_states = [state_h_enc, state_c_enc]
encoder_model = Model(encoder_inputs, encoder_states)

latent_dim = 256
decoder_state_input_hidden = Input(shape=(latent_dim,))
decoder_state_input_cell = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_hidden, decoder_state_input_cell]
decoder_outputs, state_hidden, state_cell = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_hidden, state_cell]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

def decode_response(test_input):
    #Getting the output states to pass into the decoder
    states_value = encoder_model.predict(test_input)
    #Generating empty target sequence of length 1
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    #Setting the first token of target sequence with the start token
    target_seq[0, 0, target_features_dict['<START>']] = 1.
    
    #A variable to store our response word by word
    decoded_sentence = ''
    
    stop_condition = False
while not stop_condition:
      #Predicting output tokens with probabilities and states
      output_tokens, hidden_state, cell_state = decoder_model.predict([target_seq] + states_value)
#Choosing the one with highest probability
      sampled_token_index = np.argmax(output_tokens[0, -1, :])
      sampled_token = reverse_target_features_dict[sampled_token_index]
      decoded_sentence += " " + sampled_token
#Stop if hit max length or found the stop token
      if (sampled_token == '<END>' or len(decoded_sentence) > max_decoder_seq_length):
        stop_condition = True
#Update the target sequence
      target_seq = np.zeros((1, 1, num_decoder_tokens))
      target_seq[0, 0, sampled_token_index] = 1.
      #Update states
      states_value = [hidden_state, cell_state]
return decoded_sentence

In [ ]:
class ChatBot:
  negative_responses = ("no", "negativo", "nada", "nop", "nopi")
  exit_commands = ("chao", "adios", "nos vemos",
                 "suerte", "chaito", "bye", "chaolin")
#Method to start the conversation
  def start_chat(self):
    user_response = input("Hola, bienvenido en que te puedo ayudar?\n")
    
    if user_response in self.negative_responses:
      print("Ok, perfecto, que tengas un hermoso día!")
      return
    self.chat(user_response)
#Method to handle the conversation
  def chat(self, reply):
    while not self.make_exit(reply):
      reply = input(self.generate_response(reply)+"\n")
    
  #Method to convert user input into a matrix
  def string_to_matrix(self, user_input):
    tokens = re.findall(r"[\w']+|[^\s\w]", user_input)
    user_input_matrix = np.zeros(
      (1, max_encoder_seq_length, num_encoder_tokens),
      dtype='float32')
    for timestep, token in enumerate(tokens):
      if token in input_features_dict:
        user_input_matrix[0, timestep, input_features_dict[token]] = 1.
    return user_input_matrix
  
  #Method that will create a response using seq2seq model we built
  def generate_response(self, user_input):
    input_matrix = self.string_to_matrix(user_input)
    chatbot_response = decode_response(input_matrix)
    #Remove <START> and <END> tokens from chatbot_response
    chatbot_response = chatbot_response.replace("<START>",'')
    chatbot_response = chatbot_response.replace("<END>",'')
    return chatbot_response
#Method to check for exit commands
  def make_exit(self, reply):
    for exit_command in self.exit_commands:
      if exit_command in reply:
        print("Que tengas un hermoso día!")
        return True
    return False
  
chatbot = ChatBot()
chatbot.start_chat()

In [ ]:
chatbot.start_chat()